# Evaluation and Visualisation of Results on PLAsTiCC with `snx`


In [ ]:
import argparse
import joblib
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
import seaborn as sns
import tensorflow as tf

from itertools import cycle
from numpy import interp
from pathlib import Path
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from tensorflow import keras

from astronet.constants import astronet_working_directory as asnwd
from astronet.preprocess import one_hot_encode
from astronet.utils import astronet_logger, load_dataset

In [ ]:
from astronet.metrics import WeightedLogLoss
from astronet.visualise_results import plot_acc_history, plot_confusion_matrix, plot_loss_history, plot_multiROC

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

import random as python_random
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
python_random.seed(RANDOM_SEED)

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": ["Computer Modern Roman"]})

mpl.style.use("seaborn")

In [ ]:
architecture = "snX"

In [ ]:
dataset = "plasticc"

In [ ]:
X_train, y_train, X_test, y_test, loss = load_dataset(dataset)

In [ ]:
num_classes = y_train.shape[1]
num_classes

In [ ]:
with open(f"{asnwd}/astronet/{architecture}/models/{dataset}/results.json") as f:
    events = json.load(f)
    # Get params for best model with lowest loss
    event = min(events['training_result'], key=lambda ev: ev['loss'])

In [ ]:
# Show short summary of items in 'event'
dict(list(event.items())[1:4])

In [ ]:
model_name = event['name']

In [ ]:
plot_acc_history(dataset, model_name, event, save=False)

In [ ]:
plot_loss_history(dataset, model_name, event, save=False)

In [ ]:
model = keras.models.load_model(f"{asnwd}/astronet/{architecture}/models/{dataset}/model-{model_name}", 
                                custom_objects={'WeightedLogLoss': WeightedLogLoss()}, 
                               compile=False)

In [ ]:
with open(f"{asnwd}/data/{dataset}.encoding", "rb") as eb:
    encoding = joblib.load(eb)
class_encoding = encoding.categories_[0]

if dataset == "plasticc":
    class_mapping = {
        15: "TDE",
        42: "SNII",
        52: "SNIax",
        62: "SNIbc",
        64: "KN",
        67: "SNIa-91bg",
        88: "AGN",
        90: "SNIa",
        95: "SLSN-I",
        1: "SNIa",
        2: "SNII",
        3: "SNIbc",
    }

    class_encoding
    class_names = list(np.vectorize(class_mapping.get)(class_encoding))
else:

    class_names = class_encoding

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
logloss = event["model_evaluate_on_test_loss"]
acc = event["model_evaluate_on_test_acc"]
print(logloss, acc)

wloss = WeightedLogLoss()
print(f"{wloss(y_test, y_preds).numpy():.2f}")
# Note the discreptancy seems to be down to inconsistant seeds - see 
# https://keras.io/getting_started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development

In [ ]:
plot_confusion_matrix(
    dataset,
    model_name,
    y_test,
    y_preds,
    encoding, 
    class_names,  # enc.categories_[0]
    save=False,
)

In [ ]:
plot_multiROC(dataset, model_name, model, X_test, y_test, class_names, save=False)

## `snmachine` Benchmark Comparisons

As mentioned in https://github.com/tallamjr/astronet/issues/44 one would like to compare performance with the reuslts shown below:
    
<img src="https://user-images.githubusercontent.com/8843728/101194521-04c46800-3656-11eb-9b8c-fc2f30f88cf5.png" width="800">

> The goal would be to compare with the table below, specifically **LL_test**, with no redshift (**z**) information:

From the results obtained for the `t2` model, a weighted log-loss score of $$LL_{test} = 0.93$$ as well as $$LL_{train} = 0.42$$

The AUC measurements for $AUC_{test}$ only, since we have only evaluated on the test set are: <br>
$$AUC_{test}SNIa = 0.88$$ $$AUC_{test}SNII = 0.84$$ $$AUC_{test}SNIbc = 0.88$$

**Comments**: It should be noted that from the loss vs epochs graph above, one can clearly see that a lower loss is achieable, and it is expected that this was not saved as the final model since the `EarlyStopping` criteria was _monitoring_ the `loss` instead of the `val_loss`. This can be updated which may allow for a $LL_{test} ~ 0.6$.

Furthermore, it may be seen that from the same graph, the validation loss plataeus at ~ 0.86, but the reported $LL_{test} ~ 0.93$, why? 

This seems to be a case of random seeds. This has been looked into with additions of `PYTHONHASHSEED=0` added as an environment variable. See these [Keras FAQs](https://keras.io/getting_started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development) for details.